In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.layers import Reshape 
from keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras.utils import plot_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten

In [2]:
image_path = r'C:\Users\sonuc\Desktop\Data_Science\Transpher learning and cat-and-dog SIMPLE\Transpher_learning\Method_3\utk_facee\UTKFace'

In [3]:
age = []
gender = []
img_names = []


for imagess in os.listdir(image_path):
    
    img_path = os.path.join(image_path,imagess)
    age.append(int(imagess.split('_')[0]))
    gender.append(int(imagess.split('_')[1]))
    img_names.append(imagess)

In [4]:
len(img_names)

23708

our both class is in same folder so we'll make a data frame of each class

In [5]:
df = pd.DataFrame({'Age': age, 'Gender':gender, 'Image':img_names})

In [6]:
df

,Age,Gender,Image
0,100,0,100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,100_1_0_20170112213303693.jpg.chip.jpg
...,...,...,...
23703,9,1,9_1_3_20161220222856346.jpg.chip.jpg
23704,9,1,9_1_3_20170104222949455.jpg.chip.jpg
23705,9,1,9_1_4_20170103200637399.jpg.chip.jpg
23706,9,1,9_1_4_20170103200814791.jpg.chip.jpg


In [7]:
# shuffle the DataFrame rows
train_df = df.sample(frac = 1,random_state = 0).iloc[:20000]
test_df = df.sample(frac = 1,random_state = 0).iloc[20000:]

In [8]:
train_df.shape,test_df.shape

((20000, 3), (3708, 3))

In [9]:
df.reset_index(inplace = True, drop = True)

In [10]:
resnet = ResNet50(include_top = False,input_shape = (200,200,3))

In [11]:
output = resnet.layers[-1].output
flatten = Flatten()(output)

In [12]:
dense1 = Dense(512, activation="relu")(flatten)
dense2 = Dense(512, activation="relu")(flatten)

dense3 = Dense(512, activation="relu")(dense1)
dense4 = Dense(512, activation="relu")(dense2)

In [13]:
output_regression = Dense(1, activation="linear", name = 'Age')(dense3)
output_classification = Dense(1, activation="sigmoid", name = 'Gender')(dense4)

In [14]:
resnet_model=Model(inputs= resnet.input,outputs=[output_regression,output_classification])

In [15]:
# freezing all layers of resnet50 model
for layer in resnet.layers:
    layer.trainable = False

In [16]:
#checking trainable parameter
resnet_model.summary()  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [17]:
resnet_model.compile(loss = {'Age':'mse', 'Gender': 'binary_crossentropy'},
              optimizer='Adam',
              metrics = {'Age':'mse', 'Gender': 'accuracy'})

Data_Augmention

In [18]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=image_path,  # this is the target directory
        x_col="Image",
        y_col=['Age', 'Gender'],
        target_size=(200, 200),  # all images will be resized to 200*200
        batch_size=batch_size,
        class_mode='multi_output')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=image_path,  # this is the target directory
        x_col="Image",
        y_col=['Age', 'Gender'],
        target_size=(200, 200),  # all images will be resized to 200*200
        batch_size=batch_size,
        class_mode='multi_output')  # since we use binary_crossentropy loss, we need binary labels


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [47]:
resnet_model.fit(train_generator,batch_size = 32,epochs=10,validation_data=validation_generator)
#model.save('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
625/625 [==============================] - 164s 263ms/step - loss: 223.0125 - Age_loss: 222.3202 - Gender_loss: 0.6921 - Age_mse: 222.3202 - Gender_accuracy: 0.5236 - val_loss: 239.6020 - val_Age_loss: 238.9094 - val_Gender_loss: 0.6926 - val_Age_mse: 238.9094 - val_Gender_accuracy: 0.5173
Epoch 2/10
625/625 [==============================] - 164s 263ms/step - loss: 215.5659 - Age_loss: 214.8738 - Gender_loss: 0.6921 - Age_mse: 214.8738 - Gender_accuracy: 0.5236 - val_loss: 193.6102 - val_Age_loss: 192.9176 - val_Gender_loss: 0.6926 - val_Age_mse: 192.9176 - val_Gender_accuracy: 0.5173
Epoch 3/10
625/625 [==============================] - 165s 264ms/step - loss: 216.7999 - Age_loss: 216.1082 - Gender_loss: 0.6921 - Age_mse: 216.1082 - Gender_accuracy: 0.5236 - val_loss: 243.7767 - val_Age_loss: 243.0840 - val_Gender_loss: 0.6926 - val_Age_mse: 243.0840 - val_Gender_accuracy: 0.5173
Epoch 4/10
625/625 [==============================] - 166s 266ms/step - loss: 208.0834 - Age_l

In [48]:
resnet_model.save('mymymy.h5')  # always save your weights after training or during training

C:\Users\sonuc\anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
